<a href="https://colab.research.google.com/github/wjleece/ai-reflector/blob/main/wjleece_Langchain_Reflector_2_LLMs_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -U --quiet  langchain langgraph
%pip install -U --quiet tavily-python
%pip install -U --quiet fireworks-ai
%pip install -U --quiet langchain_fireworks

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.5/113.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.7/407.7 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4

In [ ]:
import os
import getpass
import re
from langchain_fireworks import Fireworks
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import LLMChain

In [ ]:
def set_env_var(var: str) -> None:
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"Enter {var}: ")

# Set up environment variables
set_env_var("LANGCHAIN_API_KEY")
set_env_var("FIREWORKS_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Reflection"

Enter LANGCHAIN_API_KEY: ··········
Enter FIREWORKS_API_KEY: ··········


In [ ]:
# Initialize ChatFireworks
base_llm = Fireworks(
    model="accounts/fireworks/models/llama-v3p1-70b-instruct",
    temperature=0.7,
    max_tokens=16384
)

reflector_llm = Fireworks(
    model="accounts/fireworks/models/mixtral-8x7b-instruct",
    temperature=0.7,
    max_tokens=32768
)


In [ ]:
# Define prompts
essay_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an essay assistant tasked with writing excellent 5-paragraph essays. "
               "Generate the best essay possible for the user's request. "
               "If you receive feedback or a critique of the essay, incorporate the feedback and rewrite the essay. "
               "If you rewrite the essay, preface it with 'Revised Essay:' to make it clear that what follows is a revision. "
               "If you have no feedback or critiques, you do not need to rewrite the essay."),
    MessagesPlaceholder(variable_name="messages")
])

reflection_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a teacher grading an essay submission. Generate critique and recommendations for the user's submission. "
               "Provide detailed recommendations, including requests for length, depth, style, etc. "
               "Make sure to include an overall score from 0 - 100 prefaced by the indicator 'Score:' for the essay prominently as either the first line or title of the critique."
               "Make sure the score is written as a number and not as a fraction."),
    MessagesPlaceholder(variable_name="messages")
])

In [ ]:
# Create LLMChains
generate_chain = essay_prompt | base_llm

reflect_chain = reflection_prompt | reflector_llm

In [ ]:
def extract_score(reflection):
   # print("Debug: Reflection content:", reflection)

    # Look for patterns like "Score: X" or "Score: X/Y" or "X/100" or just a number as the LLM returns all types of values, even when explicitly told not to return certain formats
    score_patterns = [
        r"Score:\s*(\d+)(?:/\d+)?",
        r"(\d+)/100",
        r"^(\d+)$"
    ]

    for pattern in score_patterns:
        match = re.search(pattern, reflection, re.MULTILINE)
        if match:
            try:
                score = int(match.group(1))
             #   print("Debug: Extracted score:", score)
                return score
            except ValueError:
                print("Debug: Failed to convert matched score to integer")

    print("Debug: No valid score pattern found")
    print("Error extracting score. Using None.")
    return None

In [ ]:
def generate_essay(request, max_attempts=3, min_essay_length=50):
    for attempt in range(max_attempts):
        essay = generate_chain.invoke({"messages": [HumanMessage(content=request)]})
        words = essay.strip().split()
        if len(words) >= min_essay_length:
            return essay
        print(f"Generated essay too short ({len(words)} words). Attempt {attempt + 1}/{max_attempts}")
    print(f"Failed to generate essay of sufficient length after {max_attempts} attempts. Using best attempt.")
    return essay

In [ ]:
def generate_and_reflect_essay(request):
    essay = generate_essay(request)
    reflection_response = reflect_chain.invoke({
        "messages": [
            HumanMessage(content=request),
            AIMessage(content=essay)
        ]
    })
    score = extract_score(reflection_response)
    return request, essay, reflection_response, score

In [ ]:
request, essay, reflection, score = generate_and_reflect_essay()

In [ ]:
request

'Write an essay on the history of Canada.'

In [ ]:
essay

" \nThe history of Canada is a rich and diverse one, spanning thousands of years. From the earliest indigenous peoples to the present day, Canada has been shaped by the interactions of various cultures, empires, and nations. This essay will explore the major events and periods that have contributed to the development of Canada as we know it today.\n\nThe earliest known human presence in Canada dates back to around 15,000 years ago, with the arrival of the Paleoindians. These indigenous peoples migrated from Asia across the Bering Land Bridge and established themselves in the region. Over time, various Native American groups developed, including the Inuit, First Nations, and Métis. Each of these groups had their own distinct culture, language, and traditions, which continue to play an important role in Canadian society today.\n\nIn the early 16th century, European explorers began to arrive in Canada, with the Vikings being the first to establish a settlement. However, it was the French 

In [ ]:
type(essay)

str

In [ ]:
reflection

"\n\n---\n\nScore: 92\n\nThe essay provides a comprehensive overview of the history of Canada, from the earliest indigenous peoples to the present day. The writer demonstrates a strong understanding of the major events and periods that have contributed to the development of Canada, and presents this information in a clear and organized manner. The essay also highlights the diversity and multiculturalism of Canadian society, which is a key aspect of the country's identity.\n\nHowever, there are a few areas where the essay could be improved. First, the essay could benefit from more depth and analysis. While the writer provides a good summary of the history of Canada, they could delve deeper into some of the key issues and debates. For example, the essay could explore the impact of European colonization on the indigenous peoples of Canada in more detail.\n\nSecond, the essay could benefit from more specific examples and anecdotes. While the writer provides a good overview of the history o

In [ ]:
type(reflection)

str

In [ ]:
score

0

In [ ]:
def iterate(request, essay, reflection, score):
    revised_essay = essay
    target_score = 95
    revision_count = 0
    max_revisions = 10
    min_essay_length = 50

    while (score is None or score < target_score) and revision_count < max_revisions:
        new_essay = generate_chain.invoke({"messages": [
            HumanMessage(content=request),
            AIMessage(content=revised_essay),
            HumanMessage(content=reflection)
        ]})

        words = new_essay.strip().split()
        essay_length = len(words)

        if essay_length >= min_essay_length:
            revised_essay = new_essay
            reflection_response = reflect_chain.invoke({"messages": [
                HumanMessage(content=request),
                AIMessage(content=revised_essay)
            ]})
            reflection = reflection_response
            new_score = extract_score(reflection)

            if new_score is not None:
                score = new_score
                revision_count += 1
                print(f"Revision count: {revision_count}")
                print(f"Essay length: {essay_length}")
                print(f"Essay score: {score}\n")
            else:
                print("Failed to extract score. Retrying without incrementing revision count.")
        else:
            print(f"Generated essay too short ({essay_length} words). Retrying...")

    if revision_count == max_revisions:
        print("Maximum number of revisions reached.")

    return request, revised_essay, reflection, score

In [ ]:
def main():
    request = "Write an essay on the history of Canada."
    request, essay, reflection, original_score = generate_and_reflect_essay(request)
    print(f"Original Essay Score: {original_score}\n")
    request, revised_essay, reflection, final_score = iterate(request, essay, reflection, original_score)
    print(f"Final Essay:\n{revised_essay}")
    print(f"Final Score: {final_score}")

if __name__ == "__main__":
    main()

Original Essay Score: 85

Generated essay too short (0 words). Retrying...
Debug: No valid score pattern found
Error extracting score. Using None.
Failed to extract score. Retrying without incrementing revision count.
Generated essay too short (0 words). Retrying...
Generated essay too short (0 words). Retrying...
Generated essay too short (0 words). Retrying...
Generated essay too short (0 words). Retrying...
Generated essay too short (0 words). Retrying...
Revision count: 1
Essay length: 994
Essay score: 85

Generated essay too short (0 words). Retrying...
Generated essay too short (0 words). Retrying...
Generated essay too short (0 words). Retrying...
Generated essay too short (0 words). Retrying...
Generated essay too short (0 words). Retrying...
Generated essay too short (0 words). Retrying...
Generated essay too short (0 words). Retrying...
Generated essay too short (0 words). Retrying...
Generated essay too short (0 words). Retrying...
Generated essay too short (0 words). Retryi

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))